# Notebook 4.3 - Actors curation: duplicates

This notebook gather several checks that can be run together or independently of each other. The set of these checks helps moderators to curate Duplicated actors in the SSH Open Marketplace. 

This notebook is composed of 6 sections:

0. Requirements to run this notebook
1. Get actors 
2. Duplicated actors 
2.1 Get duplicates for actors
    2.2 Compare duplicated actors
    2.3 Merge duplicated actors


## 0 Requirements to run this notebook

This section gives all the relevant information to "interact" with the MP data.

### 0.1 libraries
*There are a number of external libraries needed to run the notebook* 

*Furthermore, a dedicated SSH Open Marketplace library - sshmarketplacelib - with customised functions has been created and can be imported using the python import commands.* 

*Below the libraries import needed to run this notebook*

In [ ]:
import pandas as pd #to manage dataframes
#import matplotlib.pyplot as plt #to create histograms and images
#import seaborn as sns #to create histograms and images
import numpy as np #to manage json objects
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel

In [ ]:
mpdata = mpd()
utils=hel.Util()
check=eva.URLCheck()

In [ ]:
df_tool_flat =mpdata.getMPItems ("toolsandservices", True)
df_publication_flat =mpdata.getMPItems ("publications", True)
df_trainingmaterials_flat =mpdata.getMPItems ("trainingmaterials", True)
df_workflows_flat =mpdata.getMPItems ("workflows", True)
df_datasets_flat =mpdata.getMPItems ("datasets", True)

## 1. Get actors

In [ ]:
df_actors_flat =mpdata.getMPItems ("actors", False)

In [ ]:
df_actors_flat.tail()

## 2. Duplicated actors
    2.1 Get duplicates for actors using *actor.name* and *actor.website* as filters
    2.2 Compare duplicated actors (optional)
    2.4 Merge duplicated actors

### 2.1 Get duplicates for actors using *actor.name* and *actor.website* as filter

In [ ]:
utils=hel.Util()
filter_attribute=['name', 'website']
df_actor_duplicates=df_actors_flat[df_actors_flat.duplicated(subset=filter_attribute, keep=False)]
dupl_actor_website=df_actor_duplicates[df_actor_duplicates['website'].notnull()].sort_values('name')

In [ ]:
print (f'Using the attributes "{filter_attribute}" as filter, there are: {dupl_actor_website.shape[0]} duplicated actors')

In [ ]:
actorwebsite_tomerge=dupl_actor_website.groupby(['name','website'])['id'].apply(list).reset_index(name='idstobemerged')

In [ ]:
actorwebsite_tomerge.count()

In [ ]:
#The number of actors with more than one duplicate
actorwebsite_tomerge[actorwebsite_tomerge.idstobemerged.map(len)>2].count()

#### _Individuating actors with the exact same name that were never attached to any items, they could be merged without individual investigation._

In [ ]:
df_actors_empty_items=pd.DataFrame()
for item in actorwebsite_tomerge.itertuples():
    allEmpty=True;
    for actorid in item.idstobemerged:
        actitems=mpdata.getItemsforActor(str(actorid))
        allEmpty=allEmpty & actitems.empty
        if(not allEmpty):
            break
    if (allEmpty):
        entry = actorwebsite_tomerge.loc[actorwebsite_tomerge['name'] == item.name]
        df_actors_empty_items=pd.concat([df_actors_empty_items, entry])

In [ ]:
df_actors_empty_items.head()

#### _Different actors with the exact same name that were never attached to the same items => comparison step and further investigation needed before deciding if merging or not._

In [ ]:
df_actors_with_different_items=pd.DataFrame()
for item in actorwebsite_tomerge.itertuples():  
    if (len (item.idstobemerged)<2):
        continue
    tempdf_sn=mpdata.getItemsforActor(str(item.idstobemerged[0])).drop_duplicates('persistentId', keep='first')
    for actorid in item.idstobemerged[1:]:
        actitems=mpdata.getItemsforActor(str(actorid)).drop_duplicates('persistentId', keep='first')
        entry = actorwebsite_tomerge.loc[actorwebsite_tomerge['name'] == item.name]
        if ((tempdf_sn.empty & (not actitems.empty)) | (not (tempdf_sn.empty) & actitems.empty)):
            df_actors_with_different_items=pd.concat([df_actors_with_different_items, entry])
            break 
        if ((not actitems.empty)):
            tre=(actitems['persistentId'].isin(tempdf_sn['persistentId'])).value_counts()
            tre_r= (tempdf_sn['persistentId'].isin(actitems['persistentId'])).value_counts()
                 
            if ((False in tre.to_dict()) | (False in tre_r.to_dict())):
                df_actors_with_different_items=pd.concat([df_actors_with_different_items, entry])
                break


In [ ]:
df_actors_with_different_items.count()

#### _Individuating actors with the exact same name that were attacheed to same items, they could be merged without individual investigation._

In [ ]:
if (not df_actors_empty_items.empty):
    dfni=pd.merge(actorwebsite_tomerge,df_actors_empty_items,on=['name', 'website'], how="outer",indicator=True)
    dfni=dfni[dfni['_merge']=='left_only']
    dfni.rename(columns = {'idstobemerged_x': 'idstobemerged'}, inplace=True)
else:
    dfni=actorwebsite_tomerge

df_actors_with_same_items=pd.DataFrame()
if (not df_actors_with_different_items.empty):
    df_actors_with_same_items=pd.merge(dfni[['name','website','idstobemerged']],df_actors_with_different_items,on=['name', 'website'], how="outer",indicator=True)
    df_actors_with_same_items=df_actors_with_same_items[df_actors_with_same_items['_merge']=='left_only']
    df_actors_with_same_items=df_actors_with_same_items[['name', 'website','idstobemerged_x']]
    df_actors_with_same_items.rename(columns = {'idstobemerged_x': 'idstobemerged'}, inplace=True)
    df_actors_with_same_items=df_actors_with_same_items[['name', 'website', 'idstobemerged']]
else:
    df_actors_with_same_items=actorwebsite_tomerge
df_actors_with_same_items.head()
#df_actors_with_same_items[['name','website']].isin(df_actors_empty_items[['name','website']]).value_counts()

In [ ]:
df_actors_with_same_items.head()

### 2.2 Compare duplicated actors

In [ ]:
#id of duplicated actors
ids=[252344, 1828]
compareitems=df_actor_duplicates[df_actor_duplicates.id.isin(ids)]

In [ ]:
compareitems

In [ ]:
css_equal="font-size:1.5rem; border: 2px solid silver;background-color: white; padding: 10px 20px"
css_diff="background-color: lightyellow;  font-size:1.5rem; border: 2px solid silver; padding: 10px 20px"

In [ ]:
#view items
showdiff = compareitems.T.style.apply(lambda x: [css_equal if ((len(utils.lists_to_list(x.values))==1) ) else css_diff for i in x],
                    axis=1)
showdiff

### 2.3 Merge items

The code above has generated three different dataframes:

<ol>
    <li>The dataframe <i>df_actors_with_same_items</i> that contains the duplicated actors that were attached to same items</li>
    <li>The dataframe <i>df_actors_empty_items</i> that contains the duplicated actors that were *never attached to any item*</li>
    <li> The dataframe <i>df_actors_with_different_items</i> that contains the duplicated actors that were never attached to the same itemsm, <b>the actors in this dataframe should be manually merged!</b></li>
</ol>
The function <i>postMergedActors </i> uses the API entry: <i>(POST) /api/actors/{id}/merge</i> to automatically merge the actors contained in dataframes 1 or 2.

In [ ]:
for item in df_actors_with_same_items.itertuples():
    print(item.idstobemerged[0], ", ".join(str (e) for e in item.idstobemerged[1:]))
    mpdata.postMergedActors(str(item.idstobemerged[0]), ", ".join(str (e) for e in item.idstobemerged[1:]))